In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import time
import re

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

In [ ]:
# URL del artículo en Wayback Machine
url = "https://web.archive.org/web/20190910025751/https://www.elespectador.com/opinion/la-descentralizacion-en-jaque-tras-escena-del-crimen-de-karina-garcia-columna-880135"

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Diccionario para meses en español
meses = {
    "Ene": "enero", "Feb": "febrero", "Mar": "marzo", "Abr": "abril",
    "May": "mayo", "Jun": "junio", "Jul": "julio", "Ago": "agosto",
    "Sep": "septiembre", "Oct": "octubre", "Nov": "noviembre", "Dic": "diciembre"
}

# 1. Extraer título
titulo = soup.find('h1').get_text(strip=True)

# 2. Extraer fecha formateada
fecha_element = soup.find('div', class_='node-post-date')
if fecha_element:
    fecha_texto = fecha_element.get_text(strip=True).split(' - ')[0]
    dia, mes_abrev, anio = fecha_texto.split()
    fecha_formateada = f"{dia} de {meses[mes_abrev]} de {anio}"
else:
    fecha_formateada = "Fecha no encontrada"

# 3. Extraer autor (texto después de "Por:")
autor_element = soup.find('span', class_='by')  # Localiza el span con "Por:"
if autor_element:
    autor = autor_element.next_sibling.strip()  # Toma el texto HERMANO siguiente al span
else:
    autor = "Autor no encontrado"

# 4. Extraer contenido limpio con párrafos (versión mejorada)
contenido_div = soup.find('div', class_='node-body')
if contenido_div:
    # Primero eliminar el div no deseado si existe
    info_node = contenido_div.find('div', class_='info_node_hide')
    if info_node:
        info_node.decompose()  # Esto elimina completamente el div y su contenido

    # Eliminar solo elementos no deseados (scripts, iframes, etc.)
    for element in contenido_div(['script', 'style', 'iframe', 'img', 'figure']):
        element.decompose()

    # Procesar cada párrafo conservando formato semántico
    parrafos = []
    for p in contenido_div.find_all('p'):
        # Extraer todo el texto del párrafo incluyendo etiquetas de formato
        texto_parrafo = p.get_text(' ', strip=True)  # El espacio une elementos separados
        if texto_parrafo:
            # Limpieza final de espacios múltiples
            texto_parrafo = ' '.join(texto_parrafo.split())
            parrafos.append(texto_parrafo)

    contenido = '\n\n'.join(parrafos)
else:
    contenido = "Contenido no encontrado"

# Resultados
print(f"Título: {titulo}")
print(f"Fecha: {fecha_formateada}")
print(f"Autor: {autor}")
print("\n--- CONTENIDO ---\n")
print(contenido)

Título: La descentralización en jaque (tras escena del crimen de Karina García)
Fecha: Fecha no encontrada
Autor: Autor no encontrado

--- CONTENIDO ---

Contenido no encontrado


In [ ]:
# Usamos with para que el archivo se cierre automáticamente
with open("urls.txt", "r") as f:
    # Leemos todas las líneas y filtramos las vacías
    urls = [line.strip() for line in f if line.strip()]

# Eliminamos duplicados
urls = list(dict.fromkeys(urls))

# Diccionario para meses en español
meses = {
    "Ene": "enero", "Feb": "febrero", "Mar": "marzo", "Abr": "abril",
    "May": "mayo", "Jun": "junio", "Jul": "julio", "Ago": "agosto",
    "Sep": "septiembre", "Oct": "octubre", "Nov": "noviembre", "Dic": "diciembre"
}

# Lista para almacenar todos los resultados
datos = []
n = 0
for url in urls:
    try:
        print(f"Procesando: {url}")
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.text, 'html.parser')

        # 1. Extraer título
        titulo = soup.find('h1').get_text(strip=True) if soup.find('h1') else "Título no encontrado"

        # 2. Extraer fecha formateada
        fecha_element = soup.find('div', class_='node-post-date')
        if fecha_element:
            fecha_texto = fecha_element.get_text(strip=True).split(' - ')[0]
            try:
                dia, mes_abrev, anio = fecha_texto.split()
                fecha_formateada = f"{dia} de {meses[mes_abrev]} de {anio}"
            except:
                fecha_formateada = fecha_texto
        else:
            fecha_formateada = "Fecha no encontrada"

        # 3. Extraer autor
        autor_element = soup.find('span', class_='by')
        if autor_element:
            autor = autor_element.next_sibling.strip() if autor_element.next_sibling else "Autor no encontrado"
        else:
            autor = "Autor no encontrado"

        # 4. Extraer contenido limpio con párrafos (versión mejorada)
        contenido_div = soup.find('div', class_='node-body')
        if contenido_div:
            # Primero eliminar el div no deseado si existe
            info_node = contenido_div.find('div', class_='info_node_hide')
            if info_node:
                info_node.decompose()  # Esto elimina completamente el div y su contenido

            # Eliminar solo elementos no deseados (scripts, iframes, etc.)
            for element in contenido_div(['script', 'style', 'iframe', 'img', 'figure']):
                element.decompose()

            # Procesar cada párrafo conservando formato semántico
            parrafos = []
            for p in contenido_div.find_all('p'):
                # Extraer todo el texto del párrafo incluyendo etiquetas de formato
                texto_parrafo = p.get_text(' ', strip=True)  # El espacio une elementos separados
                if texto_parrafo:
                    # Limpieza final de espacios múltiples
                    texto_parrafo = ' '.join(texto_parrafo.split())
                    parrafos.append(texto_parrafo)

            contenido = '\n\n'.join(parrafos)
        else:
            contenido = "Contenido no encontrado"

        # Agregar a la lista de datos
        datos.append({
            'Autor': autor,
            'Fecha': fecha_formateada,
            'Título': titulo,
            'Contenido': contenido,
            'URL': url
        })
        #time.sleep(1)
    except Exception as e:
        print(f"Error procesando {url}: {str(e)}")
        datos.append({
            'Autor': f"Error: {str(e)}",
            'Fecha': "",
            'Título': "",
            'Contenido': "",
            'URL': url
        })
    n = n + 1
    if n % 20 == 0 and n < len(urls):
        print(f"Procesados: {n} de {len(urls)}")
        time.sleep(200)



# Crear DataFrame y guardar como CSV
df = pd.DataFrame(datos)

# Ordenar columnas
column_order = ['Autor', 'Fecha', 'Título', 'Contenido', 'URL']
df = df[column_order]

# Crear archivo Excel
nombre_archivo = "resultados_articulos.xlsx"
with pd.ExcelWriter(nombre_archivo, engine='openpyxl') as writer:
    df.to_excel(writer, index=False, sheet_name='Artículos')

    # Ajustar el ancho de las columnas
    worksheet = writer.sheets['Artículos']
    worksheet.column_dimensions['A'].width = 25  # Autor
    worksheet.column_dimensions['B'].width = 20  # Fecha
    worksheet.column_dimensions['C'].width = 40  # Título
    worksheet.column_dimensions['D'].width = 80  # Contenido
    worksheet.column_dimensions['E'].width = 60  # URL

print(f"\nProceso completado. Resultados guardados en {nombre_archivo}")

Procesando: https://web.archive.org/web/20180101101206/https://www.elespectador.com/opinion/cesar-gaviria-y-cesar-mondragon-columna-731046
Procesando: https://web.archive.org/web/20180101101206/https://www.elespectador.com/opinion/mas-recomendaciones-de-fin-de-ano-columna-731038
Procesando: https://web.archive.org/web/20180127005204/https://www.elespectador.com/opinion/adios-al-papel-y-la-tinta-bienvenida-la-tecnologia-al-servicio-de-los-ciudadanos-columna-735529
Procesando: https://web.archive.org/web/20180101101206/https://www.elespectador.com/opinion/sobre-la-construccion-de-paz-y-el-indulto-fujimori-columna-730787
Procesando: https://web.archive.org/web/20180102104221/https://www.elespectador.com/opinion/fajardo-para-nada-tibio-columna-731305
Procesando: https://web.archive.org/web/20180102104221/https://www.elespectador.com/opinion/macedonia-de-norte-columna-731309
Procesando: https://web.archive.org/web/20180102104221/https://www.elespectador.com/opinion/el-nacionalismo-segun-var

In [50]:
pip install requests-html

Note: you may need to restart the kernel to use updated packages.


In [53]:
pip install --force-reinstall requests-html

  Using cached requests_html-0.10.0-py3-none-any.whl.metadata (15 kB)
  Using cached pyquery-2.0.1-py3-none-any.whl.metadata (9.0 kB)
  Using cached fake_useragent-2.2.0-py3-none-any.whl.metadata (17 kB)
  Using cached parse-1.20.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached bs4-0.0.2-py2.py3-none-any.whl.metadata (411 bytes)
  Using cached pyppeteer-2.0.0-py3-none-any.whl.metadata (7.1 kB)
  Using cached pyee-11.1.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached urllib3-1.26.20-py2.py3-none-any.whl.metadata (50 kB)
  Using cached websockets-10.4-cp312-cp312-win_amd64.whl
Using cached requests_html-0.10.0-py3-none-any.whl (13 kB)
Using cached pyppeteer-2.0.0-py3-none-any.whl (82 kB)
Using cached bs4-0.0.2-py2.py3-none-any.whl (1.2 kB)
Using cached fake_useragent-2.2.0-py3-none-any.whl (161 kB)
Using cached parse-1.20.2-py2.py3-none-any.whl (20 kB)
Using cached pyquery-2.0.1-py3-none-any.whl (22 kB)
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ----

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aext-assistant-server 4.1.0 requires anaconda-cloud-auth>=0.7.1, which is not installed.
conda-repo-cli 1.0.114 requires urllib3>=2.2.2, but you have urllib3 1.26.20 which is incompatible.


In [55]:
from requests_html import HTMLSession
import pandas as pd
from time import sleep

# Iniciar sesión
session = HTMLSession()

def scrape_page(url):
    try:
        # Hacer la petición y renderizar JavaScript
        r = session.get(url)
        r.html.render(sleep=2, timeout=20)  # sleep espera 2 segundos después de cargar
        
        # Extraer datos (ajusta estos selectores según la nueva página)
        titulo = r.html.find('.Article-Title', first=True).text if r.html.find('.Article-Title') else "No encontrado"
        
        # Buscar autor - ajusta el selector según la nueva estructura
        autor = r.html.find('.author-name', first=True).text if r.html.find('.author-name') else "Anónimo"
        
        # Extraer contenido - ajusta el selector
        parrafos = [p.text for p in r.html.find('article p')] if r.html.find('article p') else []
        contenido = '\n\n'.join(parrafos) if parrafos else "Contenido no disponible"
        
        return {
            'Título': titulo,
            'Autor': autor,
            'Contenido': contenido,
            'URL': url
        }
        
    except Exception as e:
        print(f"Error en {url}: {str(e)}")
        return {
            'Título': f"Error: {str(e)}",
            'Autor': "",
            'Contenido': "",
            'URL': url
        }

# Lista de URLs (ejemplo)
urls = [
    "https://web.archive.org/web/20200609164031mp_/https://www.elespectador.com/opinion/interrumpir-el-olvido/",
]

# Procesar todas las URLs
datos = []
for i, url in enumerate(urls, 1):
    print(f"Procesando {i}/{len(urls)}: {url}")
    datos.append(scrape_page(url))
    sleep(1)  # Pausa entre solicitudes

# Guardar en Excel
df = pd.DataFrame(datos)
df
#df.to_excel('articulos_elespectador.xlsx', index=False)
#print("Scraping completado. Datos guardados en articulos_elespectador.xlsx")

Procesando 1/1: https://web.archive.org/web/20200609164031mp_/https://www.elespectador.com/opinion/interrumpir-el-olvido/
Error en https://web.archive.org/web/20200609164031mp_/https://www.elespectador.com/opinion/interrumpir-el-olvido/: Cannot use HTMLSession within an existing event loop. Use AsyncHTMLSession instead.


,Título,Autor,Contenido,URL
0,Error: Cannot use HTMLSession within an existi...,,,https://web.archive.org/web/20200609164031mp_/...


# Nuevo scrapping

In [43]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re

def setup_driver():
    """Configura el driver de Chrome optimizado"""
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--blink-settings=imagesEnabled=false")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
    driver = webdriver.Chrome(options=chrome_options)
    driver.set_page_load_timeout(30)
    return driver

def parse_date(date_str):
    """Convierte fechas al formato '1 de febrero de 2020'"""
    try:
        months_es = {
            'Jan': 'enero', 'Feb': 'febrero', 'Mar': 'marzo', 'Apr': 'abril',
            'May': 'mayo', 'Jun': 'junio', 'Jul': 'julio', 'Aug': 'agosto',
            'Sep': 'septiembre', 'Oct': 'octubre', 'Nov': 'noviembre', 'Dec': 'diciembre',
            'January': 'enero', 'February': 'febrero', 'March': 'marzo', 'April': 'abril',
            'June': 'junio', 'July': 'julio', 'August': 'agosto', 'September': 'septiembre',
            'October': 'octubre', 'November': 'noviembre', 'December': 'diciembre'
        }
        
        # Intenta parsear desde formato ISO (2020-06-01T01:16:11.578Z)
        if 'T' in date_str:
            dt = datetime.fromisoformat(date_str.replace('Z', '+00:00'))
            day = dt.day
            month = months_es[dt.strftime('%B')]
            year = dt.year
            return f"{day} de {month} de {year}"
        
        # Intenta parsear desde formato '1 Jun 2020'
        parts = re.split(r'[\s,]+', date_str.strip())
        if len(parts) >= 3:
            day = parts[0]
            month_abbr = parts[1]
            month = months_es.get(month_abbr, month_abbr)
            year = parts[2]
            return f"{day} de {month} de {year}"
            
        return date_str
    except Exception as e:
        print(f"Error parseando fecha '{date_str}': {str(e)}")
        return date_str

def extract_with_selenium(driver, url):
    """Extrae datos usando Selenium para contenido dinámico"""
    try:
        driver.get(url)
        
        # Espera inteligente con timeout extendido
        WebDriverWait(driver, 15).until(
            EC.or_(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".Article-Title")),
                EC.presence_of_element_located((By.CSS_SELECTOR, ".font--secondary"))
            )
        )

        # Extracción optimizada con JavaScript
        article_data = driver.execute_script("""
            const getText = (selector) => {
                const el = document.querySelector(selector);
                return el ? el.innerText.trim() : '';
            };
            
            const getContent = () => {
                const contentEl = document.querySelector('.font--secondary') || 
                                 document.querySelector('article');
                if (!contentEl) return '';
                
                return Array.from(contentEl.querySelectorAll('.font--secondary'))
                    .map(.font--secondary => .font--secondary.innerText.trim())
                    .filter(t => t.length > 0)
                    .join('\\n\\n');
            };
            
            return {
                title: getText('.Article-Title'),
                author: document.querySelector('.Article-Author')?.content || '',
                date: document.querySelector('.Article-Time')?.content || 
                      document.querySelector('meta[name="article:published_time"]')?.content ||
                      document.querySelector('time.Article-Time')?.innerText.split('-')[0].trim() || '',
                content: getContent()
            };
        """)
        
        return {
            'Autor': article_data.get('author', 'Autor no encontrado'),
            'Fecha': parse_date(article_data.get('date', 'Fecha no encontrada')),
            'Título': article_data.get('title', 'No encontrado'),
            'Contenido': article_data.get('content', 'Contenido no disponible'),
            'Vínculo': url
        }
        
    except Exception as e:
        print(f"Error con Selenium en {url}: {str(e)}")
        return None

def process_urls(urls_file, output_file):
    """Procesa múltiples URLs desde archivo"""
    with open(urls_file, 'r', encoding='utf-8') as f:
        urls = [line.strip() for line in f if line.strip()]
    
    driver = setup_driver()
    results = []
    
    for i, url in enumerate(urls, 1):
        print(f"Procesando ({i}/{len(urls)}): {url}")
        
        try:
            # Primero intenta con requests (más rápido)
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(url, headers=headers, timeout=15)
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Verifica si el contenido está completo
            if not soup.find('div', class_='Article-Content'):
                print("Contenido incompleto, usando Selenium...")
                raise ValueError("Contenido requiere JavaScript")
            
            # Extracción con BeautifulSoup
            title = soup.find('h1', class_='Article-Title')
            author = soup.find('meta', {'name': 'cXenseParse:author'})
            date = (soup.find('meta', {'name': 'article:modified_time'}) or 
                   soup.find('meta', {'name': 'article:published_time'}) or
                   soup.find('time', class_='Article-Time'))
            
            content_div = soup.find('div', class_='Article-Content')
            paragraphs = []
            if content_div:
                for p in content_div.find_all('p'):
                    paragraphs.append(p.get_text().strip())
            
            date_value = ''
            if date:
                if date.get('content'):
                    date_value = date['content']
                else:
                    date_value = date.get_text().split('-')[0].strip()
            
            result = {
                'Autor': author['content'] if author else 'Autor no encontrado',
                'Fecha': parse_date(date_value) if date_value else 'Fecha no encontrada',
                'Título': title.get_text().strip() if title else 'No encontrado',
                'Contenido': '\n\n'.join(paragraphs) if paragraphs else 'Contenido no disponible',
                'Vínculo': url
            }
            
        except Exception as e:
            print(f"Falló método rápido, intentando con Selenium: {str(e)}")
            result = extract_with_selenium(driver, url)
            if not result:
                result = {
                    'Autor': 'Error al extraer autor',
                    'Fecha': 'Error al extraer fecha',
                    'Título': 'Error al extraer título',
                    'Contenido': 'Error al extraer contenido',
                    'Vínculo': url
                }
        
        results.append(result)
        time.sleep(2)  # Intervalo entre requests
    
    driver.quit()
    
    # Guardar resultados en el orden solicitado
    df = pd.DataFrame(results)[['Autor', 'Fecha', 'Título', 'Contenido', 'Vínculo']]
    
    # Asegurar formato correcto en Excel
    writer = pd.ExcelWriter(output_file, engine='xlsxwriter')
    df.to_excel(writer, index=False, sheet_name='Artículos')
    
    # Ajustar el ancho de las columnas
    worksheet = writer.sheets['Artículos']
    worksheet.set_column('A:A', 30)  # Autor
    worksheet.set_column('B:B', 25)  # Fecha
    worksheet.set_column('C:C', 50)  # Título
    worksheet.set_column('D:D', 100) # Contenido
    worksheet.set_column('E:E', 60)  # Vínculo
    
    writer.close()
    print(f"\nProceso completado. Resultados guardados en {output_file}")

if __name__ == "__main__":
    process_urls('urls.txt', 'resultados_articulos.xlsx')

Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event)


Procesando (1/1): https://web.archive.org/web/20200609163417mp_/https://www.elespectador.com/opinion/educacion-pandemia-mas-inequidad/
Contenido incompleto, usando Selenium...
Falló método rápido, intentando con Selenium: Contenido requiere JavaScript
Error con Selenium en https://web.archive.org/web/20200609163417mp_/https://www.elespectador.com/opinion/educacion-pandemia-mas-inequidad/: module 'selenium.webdriver.support.expected_conditions' has no attribute 'or_'

Proceso completado. Resultados guardados en resultados_articulos.xlsx


# Para extraer links

In [34]:
import requests
from bs4 import BeautifulSoup

fecha = "20200616194558"
url = "https://web.archive.org/web/" + fecha + "/https://www.elespectador.com/opinion/"
inicio = "/web/" + fecha + "mp_/https://www.elespectador.com/"

try:
    response = requests.get(url)
    response.raise_for_status()
except requests.exceptions.RequestException as e:
    print(f"Error al hacer la petición HTTP: {e}")
    exit()

soup = BeautifulSoup(response.text, 'html.parser')

column_items = soup.find_all('li', class_='views-row')

column_links = []
base_url = "https://web.archive.org"

excluded_authors = [
    "Columnista invitado EE",
    "Cartas de los lectores",
    "Las igualadas",
    "La Pulla",
    "Antieditorial",
    "Columna del lector",
    "La Puesverdad"
]

for item in column_items:
    # Obtener el autor de manera más robusta
    author = ""
    author_div = item.find('div', class_='views-field-field-columnist')
    if author_div:
        author_link = author_div.find('a')
        if author_link:
            author = author_link.get_text(strip=True)
        else:
            author = author_div.get_text(strip=True).replace("Por: ", "")
    
    # Verificar si el autor está en la lista de exclusiones
    if any(excluded.lower() in author.lower() for excluded in excluded_authors):
        continue
    
    # Obtener el enlace de manera más segura
    title_div = item.find('div', class_='views-field-title')
    if not title_div:
        continue
        
    title_link = title_div.find('a')
    if not title_link or 'href' not in title_link.attrs:
        continue
    
    href = title_link['href']
    
    # Verificar que sea un enlace de columna válido

    #if (href.startswith(inicio) and 'columna-' in href):
    if (href.startswith(inicio) in href):
        full_url = base_url + href if not href.startswith(base_url) else href
        column_links.append(full_url)

# Eliminar duplicados manteniendo el orden
seen = set()
column_links = [x for x in column_links if not (x in seen or seen.add(x))]

print(f"\nSe encontraron {len(column_links)} enlaces de columnas válidos:")
for i, link in enumerate(column_links, 1):
    print(f"{link}")




Se encontraron 0 enlaces de columnas válidos:


In [38]:
import requests
import json
from bs4 import BeautifulSoup
import re

url = "https://web.archive.org/web/20200618142303/https://www.elespectador.com/opinion/"

try:
    response = requests.get(url)
    response.raise_for_status()
except requests.exceptions.RequestException as e:
    print(f"Error al hacer la petición HTTP: {e}")
    exit()

soup = BeautifulSoup(response.text, 'html.parser')

# Buscar el script que contiene los datos JSON
script_tag = soup.find('script', type='application/javascript')
if not script_tag:
    print("No se encontró el script con los datos JSON")
    exit()

# Extraer el JSON del script
json_data = {}
try:
    # Buscar la variable Fusion.globalContent
    match = re.search(r'Fusion\.globalContent\s*=\s*({.*?});', script_tag.string, re.DOTALL)
    if match:
        json_str = match.group(1)
        json_data = json.loads(json_str)
except (AttributeError, json.JSONDecodeError) as e:
    print(f"Error al procesar los datos JSON: {e}")
    exit()

# Lista para almacenar los enlaces válidos
column_links = []
base_url = "https://web.archive.org"

# Excluir estos tipos de contenido
excluded_types = [
    "Columnista invitado EE",
    "Cartas de los lectores",
    "Las igualadas",
    "La Pulla",
    "Antieditorial",
    "Columna del lector",
    "La Puesverdad"
]

# Procesar los elementos de contenido
if 'content_elements' in json_data:
    for element in json_data['content_elements']:
        if element.get('type') == 'story':
            # Obtener URL canónica
            canonical_url = element.get('canonical_url', '')
            
            # Verificar que sea una columna de opinión
            if canonical_url and '/opinion/' in canonical_url:
                # Construir URL de Wayback Machine
                wayback_date = "20200618142303"  # Puedes extraer esto de la URL original si es variable
                wayback_url = f"{base_url}/web/{wayback_date}mp_/{canonical_url}"
                
                # Verificar que no sea de los tipos excluidos
                title = element.get('headlines', {}).get('basic', '').lower()
                if not any(excluded.lower() in title for excluded in excluded_types):
                    column_links.append(wayback_url)

# Eliminar duplicados
column_links = list(dict.fromkeys(column_links))

# Mostrar resultados
print(f"\nSe encontraron {len(column_links)} enlaces de columnas válidos:")
for i, link in enumerate(column_links, 1):
    print(f"{i}. {link}")



Se encontraron 0 enlaces de columnas válidos:
